#### **DEPENDENCIES**

In [1]:
import cv2
import sys
import torch as pt
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive; drive.mount("/content/drive")

In [ ]:
!git clone https://github.com/adityaprakash-work/blind-cats-go-prr
%cd blind-cats-go-prr
!pip install -r requirements.txt
sys.path.append("../")

In [ ]:
from src.trainers.twin import Trainer1
from src.data.thoughtviz import ThoughtVizDepthBasic
from src.models import EEGEncoderSimple, GenericImageEncoder1, GenericLatentDecoder1

#### **DATASET**

In [2]:
thoughtviz_path = ...
images_dir = ...
dset = ThoughtVizDepthBasic(thoughtviz_path, images_dir)

In [ ]:
trn_size = int(0.8 * len(dset))
val_size = len(dset) - trn_size
trn_dset, val_dset = pt.utils.data.random_split(dset, [trn_size, val_size])
trn_dl = DataLoader(trn_dset, batch_size=32, shuffle=True)
val_dl = DataLoader(val_dset, batch_size=32, shuffle=True)

#### **MODELS**

In [ ]:
eeg_enc = EEGEncoderSimple()
img_enc = GenericImageEncoder1()
lat_dec = GenericLatentDecoder1()

#### **TENSORBOARD**

In [ ]:
%mkdir -p experiments/twin1
%load_ext tensorboard
%tensorboard --logdir experiments/twin1 --port 6006

#### **TRAINING**

In [13]:
trainer = Trainer1(eeg_enc, img_enc, lat_dec, trn_dl, val_dl, device="cuda")

In [ ]:
trainer.fire(
    max_epochs=101,
    log_rec_interval=16,
    branch_switch_interval=128,
    log_dir="./experiments/twin1",
)